In [21]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris, load_breast_cancer, load_wine
from sklearn.model_selection import train_test_split
from qiskit_machine_learning.datasets import ad_hoc_data
# iris = load_wine()
# X, Y = iris.data, iris.target
# train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2)

adhoc_dimension = 3
train_x, train_y, test_x, test_y, adhoc_total = ad_hoc_data(
    training_size=200,
    test_size=50,
    n=adhoc_dimension,
    gap=0.3,
    plot_data=False,
    one_hot=False,
    include_sample_total=True,
)

In [22]:
from qiskit.circuit.library import ZZFeatureMap

num_qubits = train_x.shape[1]
feature_map = ZZFeatureMap(feature_dimension=adhoc_dimension, reps=2, entanglement="linear")


In [23]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=12)
# train_x = pca.fit_transform(train_x)
# test_x = pca.transform(test_x)
# train_x.shape

In [ ]:
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit.primitives import Sampler
from sklearn.svm import SVC
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_aer import AerSimulator
# Set the SVC algorithm to use our custom kernel
sampler = Sampler()

fidelity = ComputeUncompute(sampler=sampler)

adhoc_kernel = FidelityQuantumKernel(feature_map=feature_map, fidelity=fidelity)
quantum_svc = SVC(kernel=adhoc_kernel.evaluate)
quantum_svc.fit(train_x, train_y)
x = quantum_svc.score(test_x, test_y)



normal_svc = SVC(kernel="linear")
normal_svc.fit(train_x, train_y)
y = normal_svc.score(test_x, test_y)

print(x,y)